In [31]:
from transformers import ElectraTokenizer, ElectraForQuestionAnswering#, pipelines
from pprint import pprint

In [32]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig
from transformers import ElectraModel, ElectraTokenizer
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [33]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [53]:
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model

model_path = 'bert-base-uncased'

# load model
#model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
#model.to(device)
#model.eval()
#model.zero_grad()




# load tokenizer
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
model = ElectraForQuestionAnswering.from_pretrained("google/electra-small-discriminator")
model.to(device)
model.eval()
model.zero_grad()

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['qa_outputs.weight', 'qa_output

In [54]:
model.electra.embeddings

ElectraEmbeddings(
  (word_embeddings): Embedding(30522, 128, padding_idx=0)
  (position_embeddings): Embedding(512, 128)
  (token_type_embeddings): Embedding(2, 128)
  (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [55]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    return model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )

In [56]:
def squad_pos_forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None, position=0):
    pred = predict(inputs,
                   token_type_ids=token_type_ids,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    pred = pred[position]
    return pred.max(1).values

In [57]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence
cls_token_id

101

In [58]:
def construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id):
    question_ids = tokenizer.encode(question, add_special_tokens=False)
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + question_ids + [sep_token_id] + text_ids + [sep_token_id]

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(question_ids) + [sep_token_id] + \
        [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(question_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def construct_bert_sub_embedding(input_ids, ref_input_ids,
                                   token_type_ids, ref_token_type_ids,
                                   position_ids, ref_position_ids):
    input_embeddings = interpretable_embedding1.indices_to_embeddings(input_ids)
    ref_input_embeddings = interpretable_embedding1.indices_to_embeddings(ref_input_ids)

    input_embeddings_token_type = interpretable_embedding2.indices_to_embeddings(token_type_ids)
    ref_input_embeddings_token_type = interpretable_embedding2.indices_to_embeddings(ref_token_type_ids)

    input_embeddings_position_ids = interpretable_embedding3.indices_to_embeddings(position_ids)
    ref_input_embeddings_position_ids = interpretable_embedding3.indices_to_embeddings(ref_position_ids)
    
    return (input_embeddings, ref_input_embeddings), \
           (input_embeddings_token_type, ref_input_embeddings_token_type), \
           (input_embeddings_position_ids, ref_input_embeddings_position_ids)
    
def construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                    token_type_ids=None, ref_token_type_ids=None, \
                                    position_ids=None, ref_position_ids=None):
    input_embeddings = interpretable_embedding.indices_to_embeddings(input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    ref_input_embeddings = interpretable_embedding.indices_to_embeddings(ref_input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    
    return input_embeddings, ref_input_embeddings



In [77]:
# question, text = "What is important to us?", "It is important to us to include, empower and support humans of all kinds."

question= "What does Cuomo describe about gun violence?"

text= r'The British Broadcasting Corporation (BBC) stated that Cuomo said when signing the executive order that 51 shootings occurred in New York State during the weekend celebrating American Independence Day. The executive order will provide US$138.7 million for gun violence intervention and prevention programs. Of funds. According to the report, Cuomo also described gun violence as a public health crisis that must be taken seriously. The state of emergency statement will make the state pay full attention to the crisis and give relevant resources. "If you look at the recent data, there are more people who die from gun violence and crime than from the new crown," Cuomo said. "Just like we did in response to the epidemic, New York will once again lead the nation in comprehensive Combat and prevent gun violence.” The US media previously quoted data from the US Gun Violence Archives website and reported that at least 140 people in the United States died from gun violence around Independence Day on July 4 this year. In addition, the New York City police said that a total of 21 shootings occurred in the city on the 2nd, 3rd, and 4th, with a total of 26 victims. On Independence Day, there were 12 shooting incidents in New York City with 13 victims, which was higher than the 8 shooting incidents and 8 victims on the same day last year.'

In [78]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)
tokenizer.decode(input_ids[0])

'[CLS] what does cuomo describe about gun violence? [SEP] the british broadcasting corporation ( bbc ) stated that cuomo said when signing the executive order that 51 shootings occurred in new york state during the weekend celebrating american independence day. the executive order will provide us $ 138. 7 million for gun violence intervention and prevention programs. of funds. according to the report, cuomo also described gun violence as a public health crisis that must be taken seriously. the state of emergency statement will make the state pay full attention to the crisis and give relevant resources. " if you look at the recent data, there are more people who die from gun violence and crime than from the new crown, " cuomo said. " just like we did in response to the epidemic, new york will once again lead the nation in comprehensive combat and prevent gun violence. ” the us media previously quoted data from the us gun violence archives website and reported that at least 140 people in

In [80]:
ground_truth = "violence"#"문재인 대통령"'to include, empower and support humans of all kinds'

ground_truth_tokens = tokenizer.encode(ground_truth, add_special_tokens=False)
print(ground_truth_tokens)
ground_truth_end_ind = indices.index(ground_truth_tokens[-1])
ground_truth_start_ind = ground_truth_end_ind - len(ground_truth_tokens) + 1

[4808]


In [81]:
tokenizer.decode([3337,340])

'boys [unused335]'

In [82]:
#start_scores, end_scores = predict(input_ids, \
#                                   token_type_ids=token_type_ids, \
#                                   position_ids=position_ids, \
#                                   attention_mask=attention_mask)
#print(all_tokens)
a=predict(input_ids,token_type_ids=token_type_ids,position_ids=position_ids,attention_mask=attention_mask)
#print(a.loss)
start_scores=a.start_logits
end_scores=a.end_logits


print('Question: ', question)
print('Predicted Answer: ', ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))

Question:  What does Cuomo describe about gun violence?
Predicted Answer:  of 21 shooting ##s occurred in the city on the 2nd , 3rd , and 4th , with


In [83]:
lig = LayerIntegratedGradients(squad_pos_forward_func, model.electra.embeddings)

attributions_start, delta_start = lig.attribute(inputs=input_ids,
                                  baselines=ref_input_ids,
                                  additional_forward_args=(token_type_ids, position_ids, attention_mask, 0),
                                  return_convergence_delta=True)
attributions_end, delta_end = lig.attribute(inputs=input_ids, baselines=ref_input_ids,
                                additional_forward_args=(token_type_ids, position_ids, attention_mask, 1),
                                return_convergence_delta=True)

def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

attributions_start_sum = summarize_attributions(attributions_start)
attributions_end_sum = summarize_attributions(attributions_end)

# storing couple samples in an array for visualization purposes
start_position_vis = viz.VisualizationDataRecord(
                        attributions_start_sum,
                        torch.max(torch.softmax(start_scores[0], dim=0)),
                        torch.argmax(start_scores),
                        torch.argmax(start_scores),
                        str(ground_truth_start_ind),
                        attributions_start_sum.sum(),       
                        all_tokens,
                        delta_start)

end_position_vis = viz.VisualizationDataRecord(
                        attributions_end_sum,
                        torch.max(torch.softmax(end_scores[0], dim=0)),
                        torch.argmax(end_scores),
                        torch.argmax(end_scores),
                        str(ground_truth_end_ind),
                        attributions_end_sum.sum(),       
                        all_tokens,
                        delta_end)

print('\033[1m', 'Visualizations For Start Position', '\033[0m')
viz.visualize_text([start_position_vis])

print('\033[1m', 'Visualizations For End Position', '\033[0m')
viz.visualize_text([end_position_vis])

 Visualizations For Start Position 


 Visualizations For End Position 


In [84]:
from IPython.display import Image
Image(filename='img/bert/visuals_of_start_end_predictions.png')

FileNotFoundError: [Errno 2] No such file or directory: 'img/bert/visuals_of_start_end_predictions.png'